In [49]:
import keras.backend as K
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

import cnn_builder as cbuild
import cnn_methods as cfunc
import config
import csv
import helper_fxns as hf
import importlib
import matplotlib.pyplot as plt
import numpy as np
import operator
import os
import pandas as pd
import random
%matplotlib inline

In [50]:
importlib.reload(config)
importlib.reload(cfunc)
importlib.reload(hf)
importlib.reload(cbuild)
C = config.Config()

In [51]:
model = cbuild.build_cnn(C, 'adam')#Adam(lr=0.01))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_73 (InputLayer)            (None, 36, 36, 12, 1) 0                                            
____________________________________________________________________________________________________
input_74 (InputLayer)            (None, 36, 36, 12, 1) 0                                            
____________________________________________________________________________________________________
input_75 (InputLayer)            (None, 36, 36, 12, 1) 0                                            
____________________________________________________________________________________________________
conv3d_61 (Conv3D)               (None, 34, 34, 11, 64 1216        input_73[0][0]                   
___________________________________________________________________________________________

model = cbuild.run_cnn(model, C)

In [52]:
nb_classes = len(C.classes_to_include)
voi_df = pd.read_csv(C.art_voi_path)
intensity_df = pd.read_csv(C.int_df_path)
#intensity_df.loc[intensity_df["art_int"] == 0, "art_int"] = np.mean(intensity_df[intensity_df["art_int"] > 0]["art_int"])
#intensity_df.loc[intensity_df["ven_int"] == 0, "ven_int"] = np.mean(intensity_df[intensity_df["ven_int"] > 0]["ven_int"])
#intensity_df.loc[intensity_df["eq_int"] == 0, "eq_int"] = np.mean(intensity_df[intensity_df["eq_int"] > 0]["eq_int"])

orig_data_dict, num_samples = cfunc.collect_unaug_data(C, voi_df, intensity_df)
print(num_samples)

avg_X2 = {}
for cls in C.classes_to_include:
    avg_X2[cls] = np.mean(orig_data_dict[cls][1], axis=0)

{'hemangioma': 56, 'cholangio': 56, 'cyst': 75, 'colorectal': 74, 'hcc': 87, 'fnh': 61}


In [54]:
train_ids = {} #filenames of training set originals
test_ids = {} #filenames of test set
X_test = []
X2_test = []
Y_test = []
Z_test = []
X_train_orig = []
X2_train_orig = []
Y_train_orig = []
Z_train_orig = []

train_samples = {}

for cls_num, cls in enumerate(orig_data_dict):
    cls_num = C.classes_to_include.index(cls)

    train_samples[cls] = round(num_samples[cls]*C.train_frac)

    order = np.random.permutation(list(range(num_samples[cls])))
    train_ids[cls] = list(orig_data_dict[cls][2][order[:train_samples[cls]]])
    test_ids[cls] = list(orig_data_dict[cls][2][order[train_samples[cls]:]])

    X_test = X_test + list(orig_data_dict[cls][0][order[train_samples[cls]:]])
    X2_test = X2_test + list(orig_data_dict[cls][1][order[train_samples[cls]:]])
    Y_test = Y_test + [[0] * cls_num + [1] + [0] * (nb_classes - cls_num - 1)] * \
                        (num_samples[cls] - train_samples[cls])
    Z_test = Z_test + test_ids[cls]

    X_train_orig = X_train_orig + list(orig_data_dict[cls][0][order[:train_samples[cls]]])
    X2_train_orig = X2_train_orig + list(orig_data_dict[cls][1][order[:train_samples[cls]]])
    Y_train_orig = Y_train_orig + [[0] * cls_num + [1] + [0] * (nb_classes - cls_num - 1)] * \
                        (train_samples[cls])
    Z_train_orig = Z_train_orig + train_ids[cls]

    print("%s has %d samples for training (%d after augmentation) and %d for testing" %
          (cls, train_samples[cls], train_samples[cls] * C.aug_factor, num_samples[cls] - train_samples[cls]))

#Y_test = np_utils.to_categorical(Y_test, nb_classes)
#Y_train_orig = np_utils.to_categorical(Y_train_orig, nb_classes)
X_test = [np.array(X_test), np.array(X2_test)]
X_train_orig = [np.array(X_train_orig), np.array(X2_train_orig)]

Y_test = np.array(Y_test)
Y_train_orig = np.array(Y_train_orig)

Z_test = np.array(Z_test)
Z_train_orig = np.array(Z_train_orig)

X_test = cfunc.separate_phases(X_test)
X_train_orig = cfunc.separate_phases(X_train_orig)

hemangioma has 45 samples for training (2250 after augmentation) and 11 for testing
cholangio has 45 samples for training (2250 after augmentation) and 11 for testing
cyst has 60 samples for training (3000 after augmentation) and 15 for testing
colorectal has 59 samples for training (2950 after augmentation) and 15 for testing
hcc has 70 samples for training (3500 after augmentation) and 17 for testing
fnh has 49 samples for training (2450 after augmentation) and 12 for testing


train_generator = train_generator_func()
model_pretrain.fit_generator(train_generator, steps_per_epoch=120, epochs=50)#, callbacks=[early_stopping])

In [64]:
#early_stopping = EarlyStopping(monitor='acc', min_delta=0.01, patience=4)
train_generator = cbuild.train_generator_func(C, train_ids, intensity_df, voi_df, avg_X2, n=6, n_art=0)
model.fit_generator(train_generator, steps_per_epoch=50, epochs=100)#, callbacks=[early_stopping])

Epoch 1/100
50/50 [==============================] - 12s - loss: 0.4895 - acc: 0.8194    
Epoch 2/100
50/50 [==============================] - 11s - loss: 0.5108 - acc: 0.7933    
Epoch 3/100
50/50 [==============================] - 11s - loss: 0.4448 - acc: 0.8278    
Epoch 4/100
50/50 [==============================] - 11s - loss: 0.4708 - acc: 0.8206    
Epoch 5/100
50/50 [==============================] - 11s - loss: 0.4379 - acc: 0.8311    
Epoch 6/100
50/50 [==============================] - 10s - loss: 0.4463 - acc: 0.8194    
Epoch 7/100
50/50 [==============================] - 10s - loss: 0.4551 - acc: 0.8111    
Epoch 8/100
50/50 [==============================] - 10s - loss: 0.4571 - acc: 0.8289    
Epoch 9/100
50/50 [==============================] - 10s - loss: 0.4041 - acc: 0.8422    - ETA: 6
Epoch 10/100
50/50 [==============================] - 10s - loss: 0.3972 - acc: 0.8456    
Epoch 11/100
50/50 [==============================] - 10s - loss: 0.4162 - acc: 0.8389    


50/50 [==============================] - 10s - loss: 0.1945 - acc: 0.9261    
Epoch 91/100
50/50 [==============================] - 10s - loss: 0.1743 - acc: 0.9339    
Epoch 92/100
50/50 [==============================] - 10s - loss: 0.1944 - acc: 0.9356    
Epoch 93/100
50/50 [==============================] - 10s - loss: 0.1904 - acc: 0.9278    
Epoch 94/100
50/50 [==============================] - 10s - loss: 0.1740 - acc: 0.9261    
Epoch 95/100
50/50 [==============================] - 10s - loss: 0.2013 - acc: 0.9244    
Epoch 96/100
50/50 [==============================] - 10s - loss: 0.2004 - acc: 0.9200    
Epoch 97/100
50/50 [==============================] - 10s - loss: 0.2069 - acc: 0.9267    
Epoch 98/100
50/50 [==============================] - 10s - loss: 0.1730 - acc: 0.9378    
Epoch 99/100
50/50 [==============================] - 10s - loss: 0.1539 - acc: 0.9411    
Epoch 100/100
50/50 [==============================] - 10s - loss: 0.2139 - acc: 0.9178    


hist = model.fit(X_train2, Y_train2, batch_size=32, epochs=200, validation_data=(X_test, Y_test), callbacks=[early_stopping])

In [56]:
voi_df_art = pd.read_csv(C.art_voi_path)
voi_df_ven = pd.read_csv(C.ven_voi_path)
voi_df_eq = pd.read_csv(C.eq_voi_path)

with open(C.small_voi_path, 'r') as csv_file:
    reader = csv.reader(csv_file)
    small_vois = dict(reader)
    
for key in small_vois:
    small_vois[key] = [int(x) for x in small_vois[key][1:-1].split(', ')]

In [ ]:
cbuild.overnight_run(C)

In [57]:
C.output_img_dir = "Z:\\OUTPUT\\12-08-3d"
C.classes_to_include

['hcc', 'cholangio', 'colorectal', 'cyst', 'hemangioma', 'fnh']

In [65]:
#plot_with_bbox(fn_list[2], cls_mapping[wrong_guesses[2]])
Y_pred = model.predict(X_test)
y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_test])
y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])

cm = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")

cfunc.save_output(Z_test, y_pred, y_true, voi_df_art, small_vois, C.classes_to_include, C)

In [63]:
Y_pred = model.predict(X_train_orig)
y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_train_orig])
y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])

cm = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")

cfunc.save_output(Z_train_orig, y_pred, y_true, voi_df_art, small_vois, C.classes_to_include, C, save_dir="Z:\\OUTPUT\\12-08-3d\\training")

In [62]:
cm

array([[14,  0,  0,  0,  0,  3],
       [ 0,  6,  3,  0,  2,  0],
       [ 0,  2, 12,  0,  1,  0],
       [ 0,  0,  1, 14,  0,  0],
       [ 0,  0,  1,  3,  7,  0],
       [ 2,  0,  0,  0,  0, 10]], dtype=int64)

In [67]:
y_true_simp, y_pred_simp, _ = cfunc.condense_cm(y_true, y_pred, C.classes_to_include)

In [68]:
accuracy_score(y_true_simp, y_pred_simp)

0.88888888888888884

In [66]:
accuracy_score(y_true, y_pred)

0.77777777777777779

4 cls: 94/94%, 86/88%, 86%/90%
5 cls: 80%/89%, 72%/86%, 84%/86%

6 cls no artificial, separate phases: 75%/84%

6 cls 2d: 77%/84%, 74%/80%
6 cls 3d: 78%/89%

fn_list = Z_test[~np.equal(y_pred, y_true)]
wrong_guesses = np.array(y_pred)[~np.equal(y_pred, y_true)]

activ = model2.predict(X_train)
#activ = model2.predict(np.expand_dims(X_train[10],axis=0))

import operator
#print("Ground truth:", [C.classes_to_include[max(enumerate(x), key=operator.itemgetter(1))[0]] for x in Y_val[::30]])
Y_ = model.predict(X_test)
print("Predictions:", [C.classes_to_include[max(enumerate(x), key=operator.itemgetter(1))[0]] + " (%.5f%% probability)" % (max(x)*100) for x in Y_])